# SensitiveDetector

**第三次课程讲**

In [1]:
!date
!jupyter nbconvert SensitiveDetector.ipynb --to html

2022年 05月 07日 星期六 21:31:15 CST

[NbConvertApp] Converting notebook SensitiveDetector.ipynb to html


[NbConvertApp] Writing 559690 bytes to SensitiveDetector.html


